# Pythonのcv2を用いた画像の取り込み

## 参考

[OpenCV-Pythonチュートリアル » OpenCVのGUI機能 » 動画を扱う](
http://labs.eecs.tottori-u.ac.jp/sd/Member/oyamada/OpenCV/html/py_tutorials/py_gui/py_video_display/py_video_display.html)

## 注意
* ```"VIDEOIO ERROR: V4L: can't open camera by index 0"``` というエラーが発生する場合は，以下のコマンドを実行してみる
    * ```$ sudo modprobe bcm2835-v4l2```
        * 起動時に自動的に設定されるようにするため，/etc/modprobe に 「bcm2835-v4l2」という行を追加しておく
    * "v412" ではなく **"v4l2"** なので注意(数字の「1」ではなく英小文字のエル)

## カメラの準備（ケーブルの取り付け）

<img src="image/camera-1.jpg">
<img src="image/camera-2.jpg">
<img src="image/camera-3.jpg">

In [ ]:
%matplotlib inline

# cv2 は opencv-pythonパッケージのインストールで利用可能
# https://pypi.org/project/opencv-python/
import cv2
import matplotlib.pyplot as plt

# カメラからのキャプチャのためのオブジェクト取得
cap = cv2.VideoCapture(0) # 0はカメラのデバイス番号

# 1フレーム読み込む
ret, frame = cap.read()

# 読み込みに成功して入れば表示する
if ret:
    # OpenCVのBGRからmatplotlibのRGBに色を変換する
    show_img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) 

    area, (l, r) = plt.subplots(ncols=2)
    l.imshow(frame)
    r.imshow(show_img)

# 解放処理
cap.release()

In [ ]:
%matplotlib nbagg
import cv2
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# 画像取り込み部分は cv2 を利用
# 画像の表示を matplotlib を用いて行う
# 'q' キーを押すと停止する

def cap_frame(c):
    # 1フレーム読み込む
    r, f = c.read()
    if r:
        # OpenCVのBGRからmatplotlibのRGBに色を変換
        f = cv2.cvtColor(f, cv2.COLOR_BGR2RGB)
    return r, f

# カメラからのキャプチャのためのオブジェクト取得
cap = cv2.VideoCapture(0) # 0はカメラのデバイス番号

im = plt.imshow(cap_frame(cap)[1])

# 更新処理
def update(i):
    is_captured, frame = cap_frame(cap)
    if is_captured:
        im.set_data(frame)
        
ani = FuncAnimation(plt.gcf(), update, interval=1)

# クローズ動作
def close(event):
    if event.key == 'q':
        # 解放処理
        plt.close(event.canvas.figure)
        cap.release()

# キーイベントによる終了動作の設定
cid = plt.gcf().canvas.mpl_connect("key_press_event", close)

# 表示開始
plt.show()



In [ ]:
!cat capture_camera_cv2.py

In [ ]:
# スクリプトの実行(ウィンドウが開く．ESCキーで終了)
!python3 capture_camera_cv2.py

# ラズパイでの画像読み込み

## 参考

http://labs.eecs.tottori-u.ac.jp/sd/Member/oyamada/OpenCV/html/py_tutorials/py_gui/py_image_display/py_image_display.html#matplotlib


    


In [ ]:
# RPiでカメラ画像を撮影（raspistill コマンド）
!raspistill -o sample.jpg

In [ ]:
# 画像を表示
!gpicview sample.jpg

In [ ]:
# RPiでコマンドで動画を撮影
!raspivid -o video.h264 -t 5000

In [ ]:
# 動画の再生
!omxplayer video.h264

In [ ]:
# picamera のインポート
# https://picamera.readthedocs.io/en/release-1.12/
# sudo apt-get install python3-picamera
import picamera

In [ ]:
# 静止画の撮影
with picamera.PiCamera() as camera:
    camera.capture('image.jpg')
    pass

In [ ]:
# カメラの設定を色々変えてみる
# https://picamera.readthedocs.io/en/release-1.12/api_camera.html
from time import sleep

with picamera.PiCamera as camera:
    # カメラのプレビューを表示
    camera.start_preview()
    sleep(1)
    # 上下・左右の反転
    camera.hflip = True
    sleep(1)
    camera.vflip = True
    sleep(1)
    camera.hflip = False
    sleep(1)
    camera.vflip = False
    sleep(1)

    # 一部切り出し
    camera.crop = (0.3, 0.3, 0.7, 0.7)
    sleep(1)
    camera.crop = (0.0, 0.0, 1.0, 1.0)
    sleep(1)
    
    # 回転
    camera.rotation = 90
    sleep(1)
    camera.rotation = 180
    sleep(1)
    camera.rotation = 270
    sleep(1)
    camera.rotation = 0    
    sleep(1)

    camera.stop_preview()
    pass

In [ ]:
# 動画の記録(例では10秒)
with picamera.PiCamera as camera:
    camera.start_preview()
    camera.start_recording('video.h264')
    picamera.wait_recording(10)
    camera.stop_preview()
    camera.stop_recording()

In [ ]:
# 写真を保存する例
import datetime
import os

# 現在時刻からファイル名，フォルダ名を決定する
now = datetime.datetime.now()
dir_name = now.strftime('%Y%m%d')
dir_path = os.getcwd()+'/'+dir_name
file_name = now.strftime('%H%M%S')

# ディレクトリがなければ作成
if not os.path.exists(dir_path):
    os.makedirs(dir_path)
    os.chmod(dir_path, 0o0777)

# 画像の撮影，保存
with picamera.PiCamera() as camera:
    camera.capture(dir_path+'/'+file_name+'.jpg')
    pass